In [2]:
from datetime import datetime
from pickle import FALSE
from pymongo import MongoClient
from tqdm import tqdm
import re
import numpy as np
from collections import Counter
import pandas as pd

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
TRIBUNAIS = ['tjmg','tjpb','trf2','stj']
SPLITS = ['test', 'validation', 'train']

In [4]:
def statistics(TRIBUNAL, SPLIT):
    mongocoll = MongoClient().juridico[TRIBUNAL+'_labeled_lines']

    num_docs = 0 
    num_pages = 0
    num_lines = 0

    doc_labels = []

    if SPLIT == 'train':
        filter = {'$or':[{'dataset':{'$exists':0}},{'dataset':'train'}]}

    else:
        filter = {'dataset':SPLIT}

    #mongocoll.find({'dataset':SPLIT},{'document.lines.tokens':0, "x" : 0, "width" : 0, "y" : 0, "height" : 0})

    for doc in mongocoll.find(filter,
                             {'document.lines.tokens':0, "document.lines.x" : 0, "document.lines.width" : 0, "document.lines.y" : 0, "document.lines.height" : 0}):
        num_docs += 1
        num_pages += len(doc['document'])

        #print([j['section_label'] for i in doc['document'] for j in i['lines']])
        page_labels = [j['section_label'] for i in doc['document'] for j in i['lines']]

        num_lines += len(page_labels)
        
        doc_labels.append(page_labels)

    classes = Counter([item for sublist in doc_labels for item in sublist])

    return{
        'tribunal':TRIBUNAL,
        'split': SPLIT,
        'num_docs':num_docs,
        'num_pages':num_pages,
        'num_lines':num_lines,
        'classes': classes
    }


In [6]:
statistics('stj', 'train')

{'tribunal': 'stj',
 'split': 'train',
 'num_docs': 500998,
 'num_pages': 4938541,
 'num_lines': 171215647,
 'classes': Counter({'None': 11333333,
          'PARTES': 9903318,
          'EMENTA': 18389733,
          'ACORDAO': 6212443,
          'RELATORIO': 20642953,
          'VOTO': 85918807,
          'CERTIDAO_DE_JULGAMENTO': 7704474,
          'AUTUACAO': 7372945,
          'CERTIDAO': 3737641})}

In [37]:
def statistics(TRIBUNAL, SPLIT):
    mongocoll = MongoClient().juridico[TRIBUNAL+'_labeled_lines']

    num_docs = 0 
    num_pages = 0
    num_lines = 0

    doc_labels = []

    if SPLIT == 'train':
        filter = {'$match' : {'dataset':{'$exists': 0}}}
    else:
        filter = {'$match' : {'dataset':SPLIT}}

    mongocoll.find({'dataset':SPLIT},{'document.lines.tokens':0})

    for doc in mongocoll.aggregate([filter,{'$project' : 
                                {'page':"$document.lines.section_label"}
                            }]):#,{'$limit' : 5 }])):
        num_docs += 1
        num_pages += len(doc['page'])
        page_labels = [item for sublist in doc['page'] for item in sublist]

        num_lines += len(page_labels)
        
        doc_labels.append(page_labels)

    classes = Counter([item for sublist in doc_labels for item in sublist])

    return{
        'tribunal':tribunal,
        'split': split,
        'num_docs':num_docs,
        'num_pages':num_pages,
        'num_lines':num_lines,
        'classes': classes
    }


In [126]:
df_geral = pd.DataFrame()
for tribunal in TRIBUNAIS:
    for split in SPLITS:
        data = (statistics(tribunal,split))
        classes = {}

        #consolida informações de sumarização de classes por tribunal.
        for i in data['classes']:
            classes.update({i: str(data['classes'][i])+'('+str(round(100*(data['classes'][i]/data['num_lines']),2))+"%)"})
        

        #concatena numeros gerais e sumarização de classes
        attr = {**{'Número de documentos': data['num_docs'],
                'Número de páginas': data['num_pages'],
                'Número de linhas': data['num_lines']},
                **classes}
        df = pd.DataFrame(attr.values(), columns=[tribunal.upper()+" "+split.upper()], index=attr.keys()).T
        df_geral = pd.concat([df_geral,df])
       
        
df_geral.fillna(0).T

Error: Canceled future for execute_request message before replies were done